In [10]:
%pip install pandas numpy networkx matplotlib scikit-learn --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install networkx --quiet
!pip install python-louvain --quiet
!pip install metis --quiet
!pip install numpy --quiet
!pip install scikit-learn --quiet
!pip install scipy --quiet
!pip install markov-clustering --quiet

In [4]:
!pip install PyMetis==2023.1.1

  error: subprocess-exited-with-error
  
  × Building wheel for PyMetis (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [9 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-310\pymetis
      copying pymetis\version.py -> build\lib.win-amd64-cpython-310\pymetis
      copying pymetis\__init__.py -> build\lib.win-amd64-cpython-310\pymetis
      running build_ext
      building 'pymetis._internal' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for PyMetis
ERROR: Could not build wheels for PyMetis, which is required to install pyproject.toml-based projects

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To up


                                              0.0/343.7 kB ? eta -:--:--
     ----                                  41.0/343.7 kB 960.0 kB/s eta 0:00:01
     ---------                             92.2/343.7 kB 871.5 kB/s eta 0:00:01
     ---------------                      143.4/343.7 kB 944.1 kB/s eta 0:00:01
     ----------------                     153.6/343.7 kB 762.6 kB/s eta 0:00:01
     ---------------------                204.8/343.7 kB 827.9 kB/s eta 0:00:01
     -----------------------------------    317.4/343.7 kB 1.1 MB/s eta 0:00:01
     -----------------------------------  337.9/343.7 kB 999.0 kB/s eta 0:00:01
     ------------------------------------ 343.7/343.7 kB 969.2 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (p

In [ ]:
import networkx as nx
import community as community_louvain
from utils.GraphConstructor import GraphConstructor
import numpy as np
from sklearn.cluster import KMeans
from scipy.sparse import csgraph
from markov_clustering import run_mcl, get_clusters
 
class CommunityDetection:
    def __init__(self, graph):
        self.graph = graph
 
    def girvan_newman(self):
        """
        Girvan-Newman algorithm for community detection.
        Returns: A list of communities.
        """
        comp = nx.algorithms.community.girvan_newman(self.graph)
        return [list(c) for c in next(comp)]
 
    def louvain(self):
        """
        Louvain algorithm for community detection.
        Returns: A dictionary of node assignments to communities.
        """
        partition = community_louvain.best_partition(self.graph.to_undirected())
        return partition
 
    # def metis_partition(self, num_partitions):
    #     """
    #     METIS partitioning algorithm.
    #     Args:
    #         num_partitions: Number of partitions to divide the graph into.
    #     Returns: A list of node assignments to communities.
    #     """
    #     _, parts = pymetis.part_graph(self.graph, nparts=num_partitions)
    #     return parts
 
    def graclus(self):
        """
        Graclus algorithm for community detection using spectral clustering as a proxy.
        Returns: A list of node assignments to communities.
        """
        laplacian = nx.normalized_laplacian_matrix(self.graph.to_undirected()).todense()
        _, vectors = np.linalg.eigh(laplacian)
        num_clusters = 3  # Default to 3 clusters; adjust as needed.
        clustering = KMeans(n_clusters=num_clusters).fit(vectors[:, :num_clusters])
        return clustering.labels_
 
    def mlr_mcl(self):
        """
        Multi-level Regularized Markov Clustering.
        Approximation using a basic Markov Clustering algorithm.
        Returns: A dictionary of node assignments to communities.
        """

 
        adj_matrix = nx.to_scipy_sparse_matrix(self.graph)
        result = run_mcl(adj_matrix)  # Run Markov Clustering
        clusters = get_clusters(result)
        community_mapping = {node: i for i, cluster in enumerate(clusters) for node in cluster}
        return community_mapping
 
    def nerstrand(self):
        """
        Nerstrand algorithm approximation using METIS partitioning.
        Returns: A list of node assignments to communities.
        """
        return self.metis_partition(num_partitions=3)
 
    def smart_local_moving(self):
        """
        Smart Local Moving (SLM) algorithm approximation using Louvain.
        Returns: A dictionary of node assignments to communities.
        """
        return self.louvain()
 
    def spectral_clustering(self, num_clusters):
        """
        Spectral clustering using the Laplacian matrix.
        Args:
            num_clusters: Number of clusters to form.
        Returns: A list of node assignments to communities.
        """
        laplacian = nx.normalized_laplacian_matrix(self.graph).todense()
        _, vectors = np.linalg.eigh(laplacian)
        clustering = KMeans(n_clusters=num_clusters).fit(vectors[:, :num_clusters])
        return clustering.labels_
 
    def label_propagation(self):
        """
        Label Propagation Algorithm (LPA) for community detection.
        Returns: A dictionary of node assignments to communities.
        """
        communities = nx.algorithms.community.label_propagation_communities(self.graph)
        return {node: i for i, community in enumerate(communities) for node in community}
 
# Esempio di utilizzo
graph_constructor = GraphConstructor(followers_path="../../../dataset/dataset_cleaned.json")
graph_constructor.build_graph()
graph = graph_constructor.graph
 
community_detector = CommunityDetection(graph)
 
# # Girvan-Newman
# girvan_newman_communities = community_detector.girvan_newman()
# print("Girvan-Newman Communities:", girvan_newman_communities)
 
# # Louvain
louvain_partition = community_detector.louvain()
print("Louvain Partition:", louvain_partition)
 
# # # METIS
# # metis_communities = community_detector.metis_partition(num_partitions=3)
# # print("METIS Communities:", metis_communities)
 
# # Graclus
graclus_communities = community_detector.graclus()
print("Graclus Communities:", graclus_communities)
 
# # MLR-MCL
# mlr_mcl_communities = community_detector.mlr_mcl()
# print("MLR-MCL Communities:", mlr_mcl_communities)
 
# # Nerstrand
# nerstrand_communities = community_detector.nerstrand()
# print("Nerstrand Communities:", nerstrand_communities)
 
# # Label Propagation
# lpa_communities = community_detector.label_propagation()
# print("Label Propagation Communities:", lpa_communities)

Louvain Partition: {62926993725: 0, 531758406: 0, 60850232106: 0, 55968501995: 0, 60628346710: 0, 65391034968: 0, 38077850387: 1, 55178050501: 1, 54579428206: 1, 64379260940: 1, 262856836: 1, 56714285656: 1, 34158564957: 1, 50656527917: 1, 62233317658: 1, 1501070805: 2, 1795971330: 2, 3098195771: 2, 33498315874: 2, 65532133383: 3, 51524079790: 3, 65242802457: 4, 58086058382: 3, 67044607463: 3, 69892071057: 3, 53982955686: 3, 62619594223: 5, 69474045690: 5, 6267328541: 36, 65465879934: 36, 67115637172: 3, 69807941929: 5, 70119456026: 3, 68982694141: 3, 65236883921: 3, 67038712330: 3, 67281319268: 3, 65360210258: 3, 67592464790: 3, 59097305524: 3, 65448076319: 3, 69511573422: 3, 69002489097: 3, 66867106474: 3, 24736383042: 7, 70690130037: 7, 49519786147: 7, 4023341285: 7, 6053618475: 7, 69822893255: 7, 63959794645: 7, 8611865700: 7, 635836312: 7, 65767865400: 7, 32882873105: 7, 4266037558: 7, 70245543127: 7, 60089084248: 7, 64568406435: 7, 4203525129: 7, 62496872860: 7, 58904352841: 7, 3

NetworkXNotImplemented: not implemented for directed type